In [1]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [149]:
%cd searia_blog

[Errno 2] No such file or directory: 'searia_blog'
/Users/damatta/searia_blog


In [177]:
data = {
    'Squad': ['Flamengo'],#, 'Palmeiras','Botafogo'], 
    'id': ['639950ae']}#, 'abdce579','d9fdd9d9']}

# data = {
#     'Squad': ['Botafogo'],#, 'Palmeiras','Botafogo'], 
#     'id': ['d9fdd9d9']}#, 'abdce579','d9fdd9d9']}

teams = pd.DataFrame(data)
teams.head()

,Squad,id
0,Flamengo,639950ae


In [192]:
#https://fbref.com/en/squads/639950ae/Flamengo-Stats
tables = ['matchlogs_for']
years = ['2024']

def fetch_team_stats(table, id, year):
    url = f"https://fbref.com/en/squads/{id}/{year}"
    res = requests.get(url)
    res.raise_for_status()
    
    soup = BeautifulSoup(res.text, "html.parser")
    html = soup.find("table", id=table)
    #read into a df
    df = pd.read_html(str(html))[0]
    
    match_ids = []
    for row in html.tbody.find_all("tr"):
        link = row.find("a", string="Match Report")
        if link:
            m = re.search(r'/en/matches/([0-9a-f]+)/', link["href"])
            match_ids.append(m.group(1))
        else:
            match_ids.append(None)  # or '' if you prefer
            
    df["match_id"] = match_ids
    

    return df


for id in teams['id']:
    team = teams.loc[teams['id'] == id, 'Squad'].iloc[0]
    print(id, team)
    for table in tables:
        for year in years:
            try:
                df = fetch_team_stats(table, id, year)
    #             print(df.head())
#                 print(f"{table}_{team}_{year}.csv")
                df.to_csv(f"{table}_{team}_{year}.csv", index=False)
            except Exception as e:
                print(f"No table found for: {table} in {id}")

In [182]:
from urllib.robotparser import RobotFileParser

rp = RobotFileParser()
rp.set_url("https://fbref.com/robots.txt")
rp.read()

url = "https://fbref.com/en/matches/a49ba761/"
if not rp.can_fetch("*", url):
    raise RuntimeError("URL disallowed by robots.txt")

In [188]:
squads = teams['Squad'].to_numpy()

tables = ['stats_639950ae_summary', 'stats_639950ae_passing', 'stats_639950ae_passing_types', 'stats_639950ae_defense',
         'stats_639950ae_possession', 'stats_639950ae_misc']

def fetch_match_stats(table, id):
    url = f"https://fbref.com/en/matches/{id}/"
    res = requests.get(url)
    res.raise_for_status()
    
    soup = BeautifulSoup(res.text, "html.parser")
    html = soup.find("table", id=table)
    #read into a df
    df = pd.read_html(str(html))[0]
                
    df["match_id"] = id
    
    df.columns = [col[1] if col[0] == "Unnamed: 0" else col[1] or col[0] for col in df.columns.values]

    return df

for squad in squads:
    
    match_ids = pd.read_csv(f"matchlogs_for_{squad}_{year}.csv")

    match_ids = match_ids[match_ids['match_id'].notna()]
    match_ids = match_ids.head(2)
#     match_ids = df.loc[3:4]
#     match_ids = df.loc[5:6]
#     match_ids = df.loc[7:8]
#     match_ids = df.loc[9:10]
#     match_ids = df.loc[11:12]

    match_ids = match_ids['match_id'].to_numpy()
#     print(match_ids)
    
    for match_id in match_ids:
        print(match_id)
        for table in tables:
            try:
                df = fetch_match_stats(table, match_id)
                df.to_csv(f"{table}_{match_id}.csv", index=False)
#                   print(f"{table}_{match_id}.csv")
            except Exception as e:
                print(f"TimedOut: {table} in {match_id}")

IndentationError: unexpected indent (3932504637.py, line 45)

In [198]:
import os
import time
import requests
import pandas as pd
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
import csv

year = 2024

# -------------------------------
# PoliteFetcher class definition
# -------------------------------
class PoliteFetcher:
    def __init__(self, base_url, user_agent, default_delay=8, cache_dir=None):
        self.base_url = base_url.rstrip('/')
        self.user_agent = user_agent
        self.default_delay = default_delay
        self.cache_dir = cache_dir
        self._init_robot_parser()

        if cache_dir:
            os.makedirs(cache_dir, exist_ok=True)

    def _init_robot_parser(self):
        robots_url = f"{self.base_url}/robots.txt"
        self.rp = RobotFileParser()
        self.rp.set_url(robots_url)
        self.rp.read()
        # Use crawl-delay if specified, else fallback
        self.delay = self.rp.crawl_delay('*') or self.default_delay

    def fetch(self, path, max_retries=10):
        """
        Fetches `base_url + path` if allowed. Honors crawl-delay,
        optionally caches pages, and retries on 429 with exponential backoff.
        """
        full_url = f"{self.base_url}{path}"
        # Check robots.txt permission
        if not self.rp.can_fetch('*', full_url):
            raise PermissionError(f"Blocked by robots.txt: {full_url}")

        # Prepare cache filename if caching enabled
        cache_path = None
        if self.cache_dir:
            safe_name = path.strip('/').replace('/', '_')
            cache_path = os.path.join(self.cache_dir, f"{safe_name}.html")
            if os.path.exists(cache_path):
                with open(cache_path, 'r', encoding='utf-8') as f:
                    return BeautifulSoup(f.read(), 'html.parser')

        headers = {'User-Agent': self.user_agent}
        wait = self.delay

        for attempt in range(1, max_retries + 1):
            resp = requests.get(full_url, headers=headers)
            if resp.status_code == 200:
                # Cache the page if required
                if cache_path:
                    with open(cache_path, 'w', encoding='utf-8') as f:
                        f.write(resp.text)
                time.sleep(self.delay)
                return BeautifulSoup(resp.text, 'html.parser')
            elif resp.status_code == 429:
                print(f"[429] Rate limited. Backing off for {wait}s (attempt {attempt})")
                time.sleep(wait)
                wait *= 2
            else:
                resp.raise_for_status()

        raise RuntimeError(f"Failed to fetch {full_url} after {max_retries} retries")

# ---------------------------------------
# Initialization and configuration block
# ---------------------------------------
# Assumes 'teams' (DataFrame with 'Squad') and 'year' are defined
# Example:
# teams = pd.read_csv("teams.csv")
# year = 2025

fetcher = PoliteFetcher(
    base_url="https://fbref.com",
    user_agent="MyAnalyticsBot/1.0 (+https://yourdomain.com/contact)",
    default_delay=8,
    cache_dir="fbref_cache"
)

tables = [
    'stats_639950ae_summary', 'stats_639950ae_passing', 'stats_639950ae_passing_types',
    'stats_639950ae_defense', 'stats_639950ae_possession', 'stats_639950ae_misc'
]

# ---------------------------------------
# Main scraping loop
# ---------------------------------------
squads = teams['Squad'].to_numpy()

for squad in squads:
    # Load match IDs for this squad
    match_logs_file = f"matchlogs_for_{squad}_{year}.csv"
    match_ids_df = pd.read_csv(match_logs_file)
    match_ids = match_ids_df['match_id'].dropna().astype(str).tolist()
    match_ids = ['471380e2']

    for match_id in match_ids:
        print(f"Processing match {match_id}")
        for table in tables:
            try:
                # Fetch and parse the page
                soup = fetcher.fetch(f"/en/matches/{match_id}/")
                html_table = soup.find("table", id=table)
                if html_table is None:
                    print(f"  Table {table} not found for match {match_id}")
                    continue

                # Read table into DataFrame
                df = pd.read_html(str(html_table))[0]
                # Add match_id column
                df["match_id"] = match_id
                df['match_id'] = df['match_id'].astype(str)
                # Clean column names
                df.columns = [
                    col[1] if col[0] == "Unnamed: 0" else col[1] or col[0]
                    for col in df.columns.values
                ]
                # Save to CSV
                out_file = f"{table}_{match_id}.csv"
                df.to_csv(out_file, index=False, quoting=csv.QUOTE_NONNUMERIC)
                print(f"  Saved {out_file}")
            except PermissionError as e:
                print(f"  PermissionError: {e}")
            except Exception as e:
                print(f"  Error fetching {table} for match {match_id}: {e}")


Processing match 471380e2
  Saved stats_639950ae_summary_471380e2.csv
  Saved stats_639950ae_passing_471380e2.csv
  Saved stats_639950ae_passing_types_471380e2.csv
  Saved stats_639950ae_defense_471380e2.csv
  Saved stats_639950ae_possession_471380e2.csv
  Saved stats_639950ae_misc_471380e2.csv


In [194]:
match_ids

['44e47b4d',
 '066baefb',
 '8aacc798',
 '6d04e942',
 '471380e2',
 '1c1074ee',
 'b4070bcd',
 '53554f9f',
 'ead21e28',
 'c8fe4906',
 '4db5fbff',
 'a49ba761']

639950ae Flamengo


In [128]:
import requests
from bs4 import BeautifulSoup

url = "https://fbref.com/en/matches/44e47b4d/"
res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text, "html.parser")

for table in soup.find_all("table"):
    print("•", table.get("id") or "(no id)")

• (no id)
• (no id)
• (no id)
• stats_639950ae_summary
• stats_639950ae_passing
• stats_639950ae_passing_types
• stats_639950ae_defense
• stats_639950ae_possession
• stats_639950ae_misc
• keeper_stats_639950ae
• stats_6f7e1f03_summary
• stats_6f7e1f03_passing
• stats_6f7e1f03_passing_types
• stats_6f7e1f03_defense
• stats_6f7e1f03_possession
• stats_6f7e1f03_misc
• keeper_stats_6f7e1f03
• shots_all
• shots_639950ae
• shots_6f7e1f03
